In [1]:
#######################################
#######################################
#######################################
#######################################
#######################################
# to run a block of code, click on the cell and press control + enter
# a block of code is still running if there's an asterisk to the left
# run this block fo code JUST ONCE
#######################################
#######################################
#######################################
#######################################

# load and optionally install required packages
if (!require('pacman')) install.packages('pacman')
pacman::p_load(
    dplyr,
    readr,
    scales,
    stringr,
    qdapRegex
)

# initial read in of all data
csv_files <- list.files(path='./data/raw/talkwalker', full.names = T, pattern = '3FoodInsecurity*')
data_groups <- lapply(csv_files, read_csv)
data <- do.call(rbind, data_groups)

# grab only columns we want and rekey some of them for convenience
data.preprocessed <- data %>%
    select(
        # post data
        content,
        domain_url,
        engagement,
        fluency_level,
        images.url,
        lang,
        matched_profile,
        noise_category,
        parent_url,
        porn_level,
        post_type,
        published,
        reach,
        sentiment,
        tags_internal,
        title,
        url,
        url_views = article_extended_attributes.url_views,
        videos.url,
        word_count,
        username = extra_author_attributes.short_name,
        
        # social media data
        facebook_followers = source_extended_attributes.facebook_followers,
        facebook_likes = article_extended_attributes.facebook_likes,
        facebook_shares = article_extended_attributes.facebook_shares,
        twitter_followers = source_extended_attributes.twitter_followers,
        twitter_likes = article_extended_attributes.twitter_likes,
        twitter_retweets = article_extended_attributes.twitter_retweets,
        twitter_shares = article_extended_attributes.twitter_shares,
        instagram_followers = source_extended_attributes.instagram_followers,
        instagram_likes = article_extended_attributes.instagram_likes,
        
        # demographic data
        author_name = extra_author_attributes.name,
        author_birthday = extra_author_attributes.birthdate.date,
        author_birthday_resolution = extra_author_attributes.birthdate.resolution,
        author_gender = extra_author_attributes.gender,
        author_short_name = extra_author_attributes.short_name,
        author_url = extra_author_attributes.url,
        author_description = extra_author_attributes.description,

        # geographic data
        
        author_continent = extra_author_attributes.world_data.continent,
        author_country = extra_author_attributes.world_data.country,
        author_country_code = extra_author_attributes.world_data.country_code,
        author_region = extra_author_attributes.world_data.region,
        author_city = extra_author_attributes.world_data.city,
        article_city = extra_article_attributes.world_data.city,
        article_latitude = extra_article_attributes.world_data.latitude,
        article_longitide = extra_article_attributes.world_data.longitude,
        source_continent = extra_source_attributes.world_data.continent,
        source_country = extra_source_attributes.world_data.country,
        source_country_code = extra_source_attributes.world_data.country_code,
        source_region = extra_source_attributes.world_data.region,
        source_city = extra_source_attributes.world_data.city
    )

# filter ot rows based on various criteria

noise_category_exclusions <- c(
    'real_estate',
    'job_offers',
    'promotions',
    'diet_pharma',
    'hate_speech',
    'seo_scam'
)

exclusion_keywords <- read_csv('./data/raw/exclusion_keywords.csv') %>%
    pull('keyword') %>%
    paste(collapse = '|')

############
## Reverse filtering
############

data.preprocessed <- data.preprocessed %>% filter(
    grepl('Food Insecurity', content, ignore.case=TRUE)
)

Loading required package: pacman

Rows: 19751 Columns: 49
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (28): url, published, content, lang, domain_url, parent_url, post_type, ...
dbl (19): porn_level, fluency_level, sentiment, article_extended_attributes....
lgl  (2): title, noise_category

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 16227 Columns: 49
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (29): url, published, content, lang, domain_url, parent_url, post_type, ...
dbl (19): porn_level, fluency_level, sentiment, article_extended_attributes....
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify 

In [2]:
#######################################
#######################################
#######################################
#######################################
#######################################
# run this block EVERY TIME you add new exclusion keywords
#######################################
#######################################
#######################################
#######################################

exclusion_keywords <- paste0(
    exclusion_keywords,
    '|',
    paste(list(
        #################################################
        #################################################
        # insert keywords in the orange area below here
        # only edit below this line
        # make sure last item is NOT followed by comma
        # one keyword per line
        # put a comma after each keyword except the last one
        #################################################
        #################################################
        "this is an example exclusion keyword",
        "exclusion keyword are case sensitive and literal",
        "Hungry groundhog",
        "losing the best girl",
        "starving for hockey",
        "starving for live big time sports,",
        "finish throwing up",
        "#SouthSudan",
        "COCKSUCKING SLAVE",
"Ayuno en Ciudad de Mexico",
"#South Sudan",
"#KikisDeliveryService",
        "i need to gts but i'm hungry",
"Hungry ticks",
"Onlyfans.com",
"#CashAppBitcoin",
"Power hungry party",
"Manuel Peter Neuer",
"#SolidaridadObrerasForenzi",
        "#China",
"#birdwatching",
"Tigray",
        "President of Hungry",
        "sit on my face",
        "hungry tax assessors",
"Chicago",
"Retail investors hungry",
"Hungry kids in the UK",
"Power hungry",
"Sahara reporters",
"Sub-Saharan Africa",
        "hungry for love",
"Hungry chute",
        "#africanow",
"Astrocytes",
"CIA just rebranded",
        "#NIGERIA",
"Starving for success",
"Government is always hungry",
        "Kraken will devour trillions",
        "Global warming",
"Power-hungry",
        "Uber Eats",
"Ghana",
"Maniac Cop 2",
"New hex bar max",
        "Riqui and Ansu",
"Tomogunchi",
        "xvids",
        "kids in Boston",
        "McDonalds is still open",
        "fans are hungry for a legit contender",
        "eat past 8",
        "yelled and cried out all my energy",
"XXXL",
"Syrian Army",
        "profit hungry",
        "I drink Prosecco",
        "Ordered food",
        "starving for a start up",
        "COVID_19uk",
        "ORDERS food thru grubhub",
        "FRONTLINE's",
        "rebuild lebanon",
        "wipe my ass",
        "hungry for a fascist",
        "decimated Beirut",
        "time for AZ to drop an album",
        "hit Venezuela hard",
        "aloha burger",
        "i just had dinner",
        "Essex",
        "money hungry",
        "Dick-hungry",
        "give us leadership",
        "huge problem in Beirut",
        "Atlanta",
"Popei’s clam bar",
        "Hungry but lazy",
        "Arctic",
        "Angola's wild",
        "#Yemen",
        "Uganda",
        "Cincinnati Food",
        "Cum hungry",
        "rural Oregon",
        "you hungry?"
        #################################################
        #################################################
        # no more edits below this line
        #################################################
        #################################################
    ), collapse = '|')
)

data.filtered <- data.preprocessed %>%
    mutate(
        content = str_remove_all(content, pattern = '@\\w+') %>% rm_url()
    ) %>%
    filter(
        !is.na(content),
        length(content) > 15,
        porn_level == 0,
        lang == 'en',
        !noise_category %in% noise_category_exclusions,
        !grepl(exclusion_keywords, content),
        !grepl('\\$SNAP', content)
    )

total_records_raw <- data %>% nrow()
total_records_filtered <- data.filtered %>% nrow()

cat(paste('total raw records:', total_records_raw, '\n'))
cat(paste('records after preprocessing:', total_records_filtered, '\n'))
cat(paste('percent records remaining:', percent(total_records_filtered / total_records_raw)))

set.seed(
    ############################
    ############################
    # change the following seed number to get a different random sample
    # you can use any number
    # run this until you get an average of 90% precision over 5 runs
    ############################
    ############################
    2028
    ############################
    ############################
    # no more edits below this line
    ############################
    ############################
)

data.filtered %>% distinct(content) %>% sample_n(10) %>% select(content)
data.filtered %>% write_csv('./data/generated/filtered_food_insecurity_3.csv');

#######################################
#######################################
#######################################
#######################################
#######################################
# read the random sample below
# calculate your own precision accuracy
# if you want to add more exclusion keywords, go back to the top of this cell, edit it, and run it again
#     you will AUTOMATICALLY get a different random sample of 10
# if you want a different sample to check for precision AND do not want to add to the exclusion keywords,
# change the seed number above and run this cell again
#######################################
#######################################
#######################################
#######################################

total raw records: 103144 
records after preprocessing: 2098 
percent records remaining: 2%

content
<chr>
"RT : “There is no excuse for a single person to go hungry in one of the wealthiest cities in the world."" Stringer has a new set of recommendations to combat food insecurity via"
"Thank you for meeting with me to discuss the urgency of acting to address starvation and hunger around the world, . Food insecurity is a major priority of our Security Council presidency, and we’ll be bringing this issue to the table on March 11."
"2020 is not just about C19. Because of conflict, #climatechange, & food insecurity 1 in 33 people around the🌎(235m) are in need of humanitarian assistance. And the most vulnerable are being hit hardest. But humanitarian response alone is not enough; we need political solutions."
Dole calls attention to food insecurity with OOH 'malnutrition labels'
"We had food insecurity before Covid19, but the virus has only made it worse. If we had a functional Republican Party we could have passed the #HeroesAct which would have helped. Mitch McConnell is still preventing any relief effort! #DemVoice1 #EveryVoice #FreshVoicesRise"
RT : NEW: Pediatricians can support patients struggling with food insecurity by connecting families to resources such as federal nutrition programs. The newly updated toolkit from and is designed to help:
"The scope of today’s “Leaders Eat Last: A Panel Series on Food Insecurity” discussion is how food insecurity affects physical outcomes, mental health, and our economy. It is imperative that we raise awareness and help our #veterans."
"RT : “I sometimes think of this as the footprint that Covid will leave on our population—increased obesity, food insecurity and chronic disease.” via"
"RT : I'm so so tired of trying to push through--pandemic, coursework, teaching, thesis, freelancing, polar vortex, apartment nonsense, food insecurity, the list goes on and on.... It's taken everything in me to survive these past two years with limited support and plot twists."
